1. Location :
2. Motivation :  
3. DataSet : 

You will choose any area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of the world that you care about. Finally, you will choose either MongoDB or SQL as the data schema to complete your project.

1. Udacity "Data Wrangling Using Mongo DB" (Lesson 6)
2. Mongo DB Docs 

1. Data Download
2. Data Audit
3. Data Cleansing/Problem Resolution
4. Load Data into MongoDB and Insights 
5. Conclusion

I can extract using the Overpass API using the Co-ordinates that are specified in the code below

In [4]:
import urllib
#for the Overpass API Git
file00 = urllib.URLopener()
file00.retrieve("http://overpass-api.de/api/map? bbox=77.3901,12.8620,77.7973,13.0815", "P3 - Data Wrangling using NoSQL/bengaluru_india.osm/bengaluru_india1.osm")

IOError: [Errno socket error] [Errno 11004] getaddrinfo failed

I can also download the predefined extract using MapZen, which can be downloaded using a link
[https://mapzen.com/data/metro-extracts/metro/bengaluru_india](https://mapzen.com/data/metro-extracts/metro/bengaluru_india/)
The downloaded data is available in this project itself

In [7]:
#Imports Master

import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections
import pymongo
from pymongo import MongoClient
import os

In [26]:
#File Master

osm_file = "P3 - Data Wrangling using NoSQL/bengaluru_india.osm/bengaluru_india.osm"
#osm_file = "P3 - Data Wrangling using NoSQL/dubai_abu-dhabi.osm/dubai_abu-dhabi.osm"
os.path.exists(osm_file)

True

In [27]:
print 'The size of the OSM file is {}'.format(os.path.getsize(osm_file)/1.0e6)

The size of the OSM file is 648.163414


In [32]:
#Using Lesson 6 Will count the no of tags

def count_tags(filename):
    tags = {}
    print filename
    for event,elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

count_tags(osm_file)

P3 - Data Wrangling using NoSQL/bengaluru_india.osm/bengaluru_india.osm


{'bounds': 1,
 'member': 6502,
 'nd': 3570566,
 'node': 2878136,
 'osm': 1,
 'relation': 964,
 'tag': 815941,
 'way': 659915}

In [33]:
#Print a sample of the OSM File
#Code sample based on Quiz of Data Wrangling using MongoDB

# Print a sample of the file. Here I am printing the first 100 elements

OSM_FILE = osm_file  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 100 # Parameter: print the first k elements (or every kth element)

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


# print the first  k top level elements
for i, element in enumerate(get_element(OSM_FILE)):
    if i <= k:
        print ET.tostring(element)
'''
# or print every  kth top level elements
for i, element in enumerate(get_element(OSM_FILE)):
    if i % k == 0:
        print ET.tostring(element)
'''
# Also writing the sample file to view it in Sublime text
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

<node changeset="16957521" id="17327077" lat="12.9026964" lon="77.5949117" timestamp="2013-07-15T08:10:50Z" uid="634020" user="user_634020" version="4" />
	
<node changeset="18611831" id="17327092" lat="12.9063367" lon="77.5950592" timestamp="2013-10-30T05:16:40Z" uid="634020" user="user_634020" version="32" />
	
<node changeset="18598983" id="17327095" lat="12.910516" lon="77.5987265" timestamp="2013-10-29T11:01:32Z" uid="634020" user="user_634020" version="32" />
	
<node changeset="44678289" id="17327106" lat="12.9171587" lon="77.5858225" timestamp="2016-12-26T09:39:52Z" uid="136860" user="indigomc" version="75">
		<tag k="name" v="Aurobindo Circle" />
		<tag k="highway" v="traffic_signals" />
		<tag k="name:kn" v="&#3205;&#3248;&#3253;&#3263;&#3202;&#3238; &#3253;&#3267;&#3236;&#3277;&#3236;" />
	</node>
	
<node changeset="833006" id="17327139" lat="12.9349712" lon="77.624083" timestamp="2009-03-19T17:09:30Z" uid="35811" user="Praveen" version="29" />
	
<node changeset="8054691" id=

import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        for val in element.iter('tag'):
            k = element.get('k')
            #search for the "lower"
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [34]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == 'tag':
        for val in element.iter('tag'):
            k = element.get('k')
            #search for the "lower"
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [36]:
sc_tag_category = process_map(osm_file)
pprint.pprint(sc_tag_category)

{'lower': 775936, 'lower_colon': 38880, 'other': 1123, 'problemchars': 2}


In [37]:
# this function will tell us how many unique users have already contributed to the map data

def get_user(element):
    if element.get('uid'):
        uid = element.attrib['uid']
        return uid
    else:
        return None
    
    

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))

    return users

contributing_users = process_map(osm_file)
len(contributing_users)

1948

So 1948 users have contributed to the map data. That's way less than what I had expected. This is a good number to start. 